# Assignment 1

#### Objectives:
1. Parse SRT Files to get drone locations
2. Find Drone images close to drone locations 

In [15]:
from libs.srtparser import parse_srt

import pandas as pd
import exifread as ef
import os 
import glob
import piexif
import numpy as  np
from scipy.spatial import distance
from math import radians, sin, cos, acos

from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

** Load the srt file **

In [2]:
csv_file='videos/DJI_0301.SRT'


In [3]:
data=parse_srt(csv_file)
times=[]
coord_x=[]
coord_y=[]
coord_z=[]
for d in data:
    times.append(d['timestamp'])
    x,y,z=[float(f) for f in d['subtitles'][0].split(',')]
    coord_x.append(x)
    coord_y.append(y)
    coord_z.append(z)

In [4]:
data_df=pd.DataFrame({'times':times,'longitude':coord_x,'latitude':coord_y})

In [5]:
data_df.head()

,latitude,longitude,times
0,19.149799,73.001358,"00:00:00,100 --> 00:00:00,200"
1,19.150096,73.001330,"00:00:00,200 --> 00:00:00,300"
2,19.150143,73.001331,"00:00:00,300 --> 00:00:00,400"
3,19.150177,73.001331,"00:00:00,400 --> 00:00:00,500"
4,19.150217,73.001324,"00:00:00,500 --> 00:00:00,600"


** Gather coordinate values from images **

In [22]:
image_dir='images/'

In [23]:
imagefile_list=glob.glob(os.path.join(image_dir,'*.JPG'))
image_names=[]
coord_x=[]
coord_y=[]

In [24]:
def get_exif_data(image):
    """Returns a dictionary from the exif data of an PIL Image item. Also converts the GPS Tags"""
    exif_data = {}
    info = image._getexif()
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            if decoded == "GPSInfo":
                gps_data = {}
                for t in value:
                    sub_decoded = GPSTAGS.get(t, t)
                    gps_data[sub_decoded] = value[t]

                exif_data[decoded] = gps_data
            else:
                exif_data[decoded] = value

    return exif_data

def _get_if_exist(data, key):
    if key in data:
        return data[key]
		
    return None
	
def _convert_to_degress(value):
    """Helper function to convert the GPS coordinates stored in the EXIF to degress in float format"""
    d0 = value[0][0]
    d1 = value[0][1]
    d = float(d0) / float(d1)

    m0 = value[1][0]
    m1 = value[1][1]
    m = float(m0) / float(m1)

    s0 = value[2][0]
    s1 = value[2][1]
    s = float(s0) / float(s1)

    return d + (m / 60.0) + (s / 3600.0)

def get_lat_lon(exif_data):
    """Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)"""
    lat = None
    lon = None

    if "GPSInfo" in exif_data:		
        gps_info = exif_data["GPSInfo"]

        gps_latitude = _get_if_exist(gps_info, "GPSLatitude")
        gps_latitude_ref = _get_if_exist(gps_info, 'GPSLatitudeRef')
        gps_longitude = _get_if_exist(gps_info, 'GPSLongitude')
        gps_longitude_ref = _get_if_exist(gps_info, 'GPSLongitudeRef')

        if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
            lat = _convert_to_degress(gps_latitude)
            if gps_latitude_ref != "N":                     
                lat = 0 - lat

            lon = _convert_to_degress(gps_longitude)
            if gps_longitude_ref != "E":
                lon = 0 - lon

    return lat, lon

In [25]:
for f in imagefile_list:
    exif_data=get_exif_data(Image.open(f))
    lat,lon=get_lat_lon(exif_data)
    coord_x.append(lon)
    coord_y.append(lat)
    image_names.append(os.path.basename(f))

In [26]:
image_df=pd.DataFrame({'image':image_names,'longitude':coord_x,'latitude':coord_y})

In [27]:
image_df.head()

,image,latitude,longitude
0,DJI_0143.JPG,19.151222,73.002145
1,DJI_0208.JPG,19.152031,73.000799
2,DJI_0111.JPG,19.150955,73.002128
3,DJI_0263.JPG,19.152560,73.001064
4,DJI_0447.JPG,19.154169,73.001840


** All latitide and longitudes in images seem to be rounded to the lowest decimal! **

** Find all given images for all given rows **

In [30]:
def distance(slat,elat,slon,elon):
    slat,elat,slon,elon=radians(slat),radians(elat),radians(slon),radians(elon)
    dist = 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))
    return dist*1000 #Convert to meters

In [39]:
image_list=image_df['image']
image_longitude=image_df['longitude']
image_latitude=image_df['latitude']
distances_all=[]
all_close_images=[]
for index,row in data_df.iterrows():
    slat=row['latitude']
    slon=row['longitude']
    images=[]
    for i in range(0,len(image_latitude)):
        elat=image_latitude[i]
        elon=image_longitude[i]
        dist=distance(slat,elat,slon,elon)
        distances_all.append(dist)
        if(dist<35):
            images.append(os.path.basename(image_list[i]))
    images=', '.join(images)
    all_close_images.append(images)

In [40]:
data_df['images']=all_close_images

In [41]:
data_df.head()

,latitude,longitude,times,images
0,19.149799,73.001358,"00:00:00,100 --> 00:00:00,200","DJI_0013.JPG, DJI_0011.JPG, DJI_0012.JPG, DJI_..."
1,19.150096,73.001330,"00:00:00,200 --> 00:00:00,300","DJI_0047.JPG, DJI_0049.JPG, DJI_0011.JPG, DJI_..."
2,19.150143,73.001331,"00:00:00,300 --> 00:00:00,400","DJI_0047.JPG, DJI_0049.JPG, DJI_0011.JPG, DJI_..."
3,19.150177,73.001331,"00:00:00,400 --> 00:00:00,500","DJI_0047.JPG, DJI_0057.JPG, DJI_0049.JPG, DJI_..."
4,19.150217,73.001324,"00:00:00,500 --> 00:00:00,600","DJI_0047.JPG, DJI_0057.JPG, DJI_0049.JPG, DJI_..."


** Save to CSV **

In [43]:
data_df.loc[:,['times','images']].to_csv('output/image_in_35m.csv')

** Times when it is close to specific places **

In [45]:
POI=pd.read_csv('assets.csv')

In [46]:
POI

,asset_name,longitude,latitude,image_names
0,Parking lot 7,73.004950,19.152112,NaN
1,Recreation centre,73.003523,19.152196,NaN
2,Canteen,73.003523,19.152196,NaN
3,Indoor heated swimming pool,73.003523,19.152343,NaN


In [60]:
images_in_50=[]
for index,row in POI.iterrows():
    images=[]
    s_lon=row['longitude']
    s_lat=row['latitude']
    for i in range(0,len(image_latitude)):
        elat=image_latitude[i]
        elon=image_longitude[i]
        dist=distance(slat,elat,slon,elon)
        if(dist<50):
            images.append(image_list[i])
    images=', '.join(images)
    images_in_50.append(images)

In [61]:
images_in_50

['', '', '', '']

** No images in range found hence avoiding saving **